In [1]:
from wmfdata import hive

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


In [3]:
hive.run("describe wmf.geoeditors_daily")

,col_name,data_type,comment
0,wiki_db,string,The wiki database of origin
1,country_code,string,The 2-letter ISO country code this group of ed...
2,user_fingerprint_or_id,string,"If an anonymous user, this is the IP + UA, oth..."
3,user_is_anonymous,boolean,Whether or not this user edited this group of ...
4,date,string,The YYYY-MM-DD date for this group of edits
5,edit_count,bigint,The total count of edits for this grouping
6,namespace_zero_edit_count,bigint,The total count of edits to namespace zero for...
7,month,string,"The month in YYYY-MM format, all edits from th..."
8,,None,None
9,# Partition Information,None,None


In [4]:
gn_countries = (
    "AD", "AL", "AT", "AX", "BA", "BE", "BG", "CH", "CY", "CZ",
    "DE", "DK", "EE", "ES", "FI", "FO", "FR", "FX", "GB", "GG",
    "GI", "GL", "GR", "HR", "HU", "IE", "IL", "IM", "IS", "IT",
    "JE", "LI", "LU", "LV", "MC", "MD", "ME", "MK", "MT", "NL",
    "NO", "PL", "PT", "RO", "RS", "RU", "SE", "SI", "SJ", "SK",
    "SM", "TR", "VA", "AU", "CA", "HK", "MO", "NZ", "JP", "SG",
    "KR", "TW", "US"
)

In [5]:
gs_edits = hive.run("""
select 
    wiki_db,
    country_code,
    sum(edit_count) as edits
from wmf.geoeditors_daily
where
    month = "{month}" and
    country_code not in {gn_countries}
group by wiki_db, country_code
""".format(
    gn_countries=repr(gn_countries),
    month="2018-08"
))

In [12]:
corrected = gs_edits.query("country_code != '--'")

In [17]:
corrected["edits"].sum()

2810226

In [14]:
corrected.groupby("country_code")["edits"].sum().sort_values(ascending=False).head(20)

country_code
IN    455534
BR    257355
UA    245489
CN    130853
IR    117767
AR    114248
MX    102353
ID     98675
VN     84951
MY     77847
PH     76181
TH     75918
CO     74001
AM     65488
CL     56734
PE     54470
BY     49807
PK     43851
AZ     38364
SA     37369
Name: edits, dtype: int64

In [15]:
corrected.groupby("wiki_db")["edits"].sum().sort_values(ascending=False).head(20)

wiki_db
enwiki          609822
eswiki          363575
wikidatawiki    271665
commonswiki     236079
ptwiki          180609
ukwiki          114743
arwiki          106929
fawiki           86414
ruwiki           77969
zhwiki           59057
viwiki           58045
idwiki           48513
thwiki           43593
hywiki           42523
frwiki           26640
azwiki           22153
mswiki           21175
hiwiki           20926
enwiktionary     20847
tewiki           20477
Name: edits, dtype: int64

In [21]:
gs_edits.query("wiki_db != 'commonswiki'")["edits"].sum()

7301671

In [22]:
corrected.query("wiki_db != 'commonswiki'")["edits"].sum()

2574147